<a href="https://colab.research.google.com/github/bilik49/taxi_case/blob/main/taxi_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Кейс про такси.

## Исходное условие задачи.

The key aspect of ride-hailing is the price. It relies on technology to collect accurate tracking data so that a fair price can be calculated at the end of the trip. Fortunately, our riders are quick to tell us when they overpaid and we can use that feedback to improve our product.

Please analyze the dataset to identify **top opportunities for reducing the 
number of overcharge tickets** and present your results. Please provide us with a PDF file/document with the findings. Finally, you shouldn’t spend more than 8 hours on this analysis.

 Variables in the file:

* order_id_new, order_try_id_new - id of an order

* calc_created- time when the order was created

* metered_price, distance, duration- actual price, distance and duration of a ride

* upfront_price- promised to the rider price, based on predicted duration (predicted_duration) and distance (predicted_distance)

* distance - ride distance

* duration - ride duration

* gps_confidence- indicator for good GPS connection (1 - good one, 0 - bad one)

* entered_by- who entered the address

* b_state- state of a ride (finished implies that the ride was actually done)

* dest_change_number- number of destination changes by a rider

* predicted distance - predicted duration of a ride based on the pickup and dropoff points entered by the rider requesting a car

* predicted duration - predicted duration of a ride based on the pickup and dropoff points entered by the rider requesting a car

* prediction_price_type- internal variable for the type of prediction:
upfront, prediction - prediction happened before the ride

* upfront_destination_changed - prediction happened after rider changed destination during the ride

* change_reason_pricing - records due to whose action the price changed

* ticket_id_new - id for customer support ticket

## Постановка задачи.

При заказе такси клиенту в приложении показывается цена, основанная на прогнозируемых времени и дистанции поездки. В конце поездки фиксируется реальная цена, основанная на фактических времени и дистанции поездки. При несоотвествии спрогнозируемой и реальной цен фиксируется переплата (работает в обе стороны). 

Необходимо найти лучшие решения для уменьшения числа переплат.

---

Загружаем данные.





In [50]:
! gdown --id 1f5_iXlAIEHCk7WPClNa5UbWjLhu7hEP8

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1f5_iXlAIEHCk7WPClNa5UbWjLhu7hEP8
To: /content/taxi_startup.csv
100% 792k/792k [00:00<00:00, 93.8MB/s]


In [51]:
import numpy as np
import pandas as pd
import scipy.stats as st
import seaborn as sns

In [52]:
df = pd.read_csv('/content/taxi_startup.csv')
df.head()

,order_id_new,order_try_id_new,calc_created,metered_price,upfront_price,distance,duration,gps_confidence,entered_by,b_state,...,device_token,rider_app_version,order_state,order_try_state,driver_app_version,driver_device_uid_new,device_name,eu_indicator,overpaid_ride_ticket,fraud_score
0,22,22,2020-02-02 3:37:31,4.04,10.0,2839,700,1,client,finished,...,NaN,CI.4.17,finished,finished,DA.4.37,1596,Xiaomi Redmi 6,1,0,-1383.0
1,618,618,2020-02-08 2:26:19,6.09,3.6,5698,493,1,client,finished,...,NaN,CA.5.43,finished,finished,DA.4.39,1578,Samsung SM-G965F,1,0,NaN
2,657,657,2020-02-08 11:50:35,4.32,3.5,4426,695,1,client,finished,...,NaN,CA.5.43,finished,finished,DA.4.37,951,Samsung SM-A530F,1,0,-166.0
3,313,313,2020-02-05 6:34:54,72871.72,NaN,49748,1400,0,client,finished,...,NaN,CA.5.23,finished,finished,DA.4.37,1587,TECNO-Y6,0,1,NaN
4,1176,1176,2020-02-13 17:31:24,20032.50,19500.0,10273,5067,1,client,finished,...,NaN,CA.5.04,finished,finished,DA.4.37,433,Itel W5504,0,0,NaN


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4943 entries, 0 to 4942
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   order_id_new           4943 non-null   int64  
 1   order_try_id_new       4943 non-null   int64  
 2   calc_created           4943 non-null   object 
 3   metered_price          4923 non-null   float64
 4   upfront_price          3409 non-null   float64
 5   distance               4943 non-null   int64  
 6   duration               4943 non-null   int64  
 7   gps_confidence         4943 non-null   int64  
 8   entered_by             4943 non-null   object 
 9   b_state                4943 non-null   object 
 10  dest_change_number     4943 non-null   int64  
 11  prediction_price_type  4923 non-null   object 
 12  predicted_distance     4923 non-null   float64
 13  predicted_duration     4923 non-null   float64
 14  change_reason_pricing  298 non-null    object 
 15  tick

In [54]:
df.isna().mean()

order_id_new             0.000000
order_try_id_new         0.000000
calc_created             0.000000
metered_price            0.004046
upfront_price            0.310338
distance                 0.000000
duration                 0.000000
gps_confidence           0.000000
entered_by               0.000000
b_state                  0.000000
dest_change_number       0.000000
prediction_price_type    0.004046
predicted_distance       0.004046
predicted_duration       0.004046
change_reason_pricing    0.939713
ticket_id_new            0.000000
device_token             1.000000
rider_app_version        0.003237
order_state              0.000000
order_try_state          0.000000
driver_app_version       0.000000
driver_device_uid_new    0.000000
device_name              0.000000
eu_indicator             0.000000
overpaid_ride_ticket     0.000000
fraud_score              0.558163
dtype: float64

Сразу избавимся от бесполезных переменных.

In [55]:
df = df.drop(['change_reason_pricing', 'device_token'], axis=1)

Посмотрим на временной промежуток данных.

In [56]:
df['calc_created'] = pd.to_datetime(df['calc_created'],format='%Y-%m-%d %H:%M:%S')
df['calc_created'].agg(['min','max'])

min   2020-02-02 00:01:16
max   2020-03-13 23:52:07
Name: calc_created, dtype: datetime64[ns]

Для решения задачи обучим модель МО и посмотрим на важность фич.  
Для этого конвертнем дату в инт и выкинем пропущенные строчки из категориальных переменных.

In [57]:
df['created_ts'] = df['calc_created'].astype(np.int64) // 10 ** 9

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  """Entry point for launching an IPython kernel.


In [58]:
df = df[(~df['prediction_price_type'].isna())&(~df['rider_app_version'].isna())].copy()

In [59]:
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
from catboost import CatBoostClassifier

In [61]:
train = df.sample(frac=0.7,random_state=49).copy()
test = df[~df.index.isin(train.index)].copy()

Проверка, что все совпало:

In [62]:
len(train)+ len(test)

4923

In [63]:
len(df)

4923

In [64]:
df.columns

Index(['order_id_new', 'order_try_id_new', 'calc_created', 'metered_price',
       'upfront_price', 'distance', 'duration', 'gps_confidence', 'entered_by',
       'b_state', 'dest_change_number', 'prediction_price_type',
       'predicted_distance', 'predicted_duration', 'ticket_id_new',
       'rider_app_version', 'order_state', 'order_try_state',
       'driver_app_version', 'driver_device_uid_new', 'device_name',
       'eu_indicator', 'overpaid_ride_ticket', 'fraud_score', 'created_ts'],
      dtype='object')

In [65]:
df.select_dtypes(include='object').columns

Index(['entered_by', 'b_state', 'prediction_price_type', 'rider_app_version',
       'order_state', 'order_try_state', 'driver_app_version', 'device_name'],
      dtype='object')

Выделяем нужные нам фичи, целевую фичу и обучаем.

In [66]:
X_features = ['order_id_new', 'order_try_id_new', 'metered_price',
       'upfront_price', 'distance', 'duration', 'gps_confidence', 'entered_by',
       'b_state', 'dest_change_number', 'prediction_price_type',
       'predicted_distance', 'predicted_duration',
       'ticket_id_new', 'rider_app_version', 'order_state',
       'order_try_state', 'driver_app_version', 'driver_device_uid_new',
       'device_name', 'eu_indicator', 'fraud_score',
       'created_ts']
y_target = ['overpaid_ride_ticket']
cat_features = df.select_dtypes(include='object').columns.to_list()

In [67]:
model = CatBoostClassifier(learning_rate=0.005, eval_metric='AUC',cat_features=cat_features, verbose=100)
model.fit(X=train[X_features],
       y=train[y_target],
       eval_set=(test[X_features],test[y_target]))

0:	test: 0.8384255	best: 0.8384255 (0)	total: 10.2ms	remaining: 10.2s
100:	test: 0.8554742	best: 0.8583712 (26)	total: 815ms	remaining: 7.25s
200:	test: 0.8609389	best: 0.8610425 (195)	total: 1.49s	remaining: 5.92s
300:	test: 0.8621228	best: 0.8621228 (300)	total: 2.35s	remaining: 5.46s
400:	test: 0.8644093	best: 0.8644537 (397)	total: 3.08s	remaining: 4.6s
500:	test: 0.8675689	best: 0.8677391 (498)	total: 4.12s	remaining: 4.1s
600:	test: 0.8701440	best: 0.8701440 (600)	total: 5.68s	remaining: 3.77s
700:	test: 0.8722011	best: 0.8722751 (699)	total: 7.6s	remaining: 3.24s
800:	test: 0.8743766	best: 0.8743766 (800)	total: 9.8s	remaining: 2.43s
900:	test: 0.8752275	best: 0.8752941 (897)	total: 11.5s	remaining: 1.26s
999:	test: 0.8760563	best: 0.8760933 (984)	total: 13.5s	remaining: 0us

bestTest = 0.8760932945
bestIteration = 984

Shrink model to first 985 iterations.


Получаем список наиболее важных фичей.  
Будем идти по списку и смотреть процент переплат и общее кол-во на сгруппированных данных.

In [68]:
fi = pd.DataFrame({'features': X_features, 'importance': model.feature_importances_})
fi.sort_values(by='importance', ascending=False)

,features,importance
20,eu_indicator,13.716023
2,metered_price,13.343260
10,prediction_price_type,8.713895
21,fraud_score,7.925970
6,gps_confidence,6.755242
17,driver_app_version,6.717570
19,device_name,5.925646
5,duration,5.762555
4,distance,5.106500
3,upfront_price,3.745148


Создадим функцию для этого.  
Если фича непрерывная величина, то будем делить ее на бины.

In [69]:
def group_target(feature):
  if (df[feature].dtype == np.float64):
    df[f'{feature}_bin'] = pd.qcut(df[feature],5)
    return df.groupby(f'{feature}_bin')['overpaid_ride_ticket'].agg(['mean','count'])
  return df.groupby(feature)['overpaid_ride_ticket'].agg(['mean','count'])

In [70]:
group_target('eu_indicator')

,mean,count
eu_indicator,,
0,0.149103,2173
1,0.004727,2750


К сожалению в описании задачи не поясняется, что значит переменная eu_indicator.  
Но она является одной из лучших метрик по выявлению переплаты (несмотря на такое огромное кол-во, процент переплат также высок)

In [71]:
group_target('metered_price')

,mean,count
metered_price_bin,,
"(1.999, 4.72]",0.000000,992
"(4.72, 8.25]",0.005112,978
"(8.25, 6000.0]",0.040279,1291
"(6000.0, 13835.1]",0.138848,677
"(13835.1, 194483.52]",0.188832,985


Видим, что при увеличении цены (дальние или длительные поездки) процент переплат растет.  
То есть существует проблема в предсказании цены дальней поездки.

In [72]:
group_target('prediction_price_type')

,mean,count
prediction_price_type,,
prediction,0.173573,1279
upfront,0.032634,3432
upfront_destination_changed,0.014423,208
upfront_waypoint_changed,0.000000,4


Также не совсем понятны значения принимаемые prediction_price_type.  
Но, несмотря на это, данная фича - хороший индикатор.

In [73]:
group_target('fraud_score')

,mean,count
fraud_score_bin,,
"(-14225.001, -1025.6]",0.013730,437
"(-1025.6, -426.0]",0.004556,439
"(-426.0, -184.0]",0.006897,435
"(-184.0, -36.0]",0.009153,437
"(-36.0, 49.0]",0.004608,434


Фича fraud_score наполовину пуста и к тому же малофинформативна для нас.  
Поэтому откажемся от нее.

In [74]:
group_target('gps_confidence')

,mean,count
gps_confidence,,
0,0.204476,983
1,0.034518,3940


При плохом уровне gps-сигнала строится неточный маршрут, что, в свою очередь, влияет на неправильную предсказанную цену.

In [75]:
group_target('device_name')

,mean,count
device_name,,
Alps F9 Pro,0.000000,10
Alps NODROPOUT T25,0.000000,1
Alps TECNO B1p,0.000000,1
Asus ASUS_A002,0.000000,5
Asus ASUS_X00TD,0.000000,3
...,...,...
"iPhone8,1",0.011236,89
"iPhone8,2",0.000000,18
"iPhone8,4",0.000000,35


Всего у нас 497 разных моделей телефонов.  
Попробуем сгруппировать device_name по производителю.

In [76]:
df['device_name'].value_counts().head(50)

TECNO MOBILE LIMITED TECNO B1p            108
HMD Global Nokia 2.2                       98
iPhone8,1                                  89
TECNO F3                                   86
HUAWEI MAR-LX1A                            72
Samsung SM-A505FN                          69
TECNO MOBILE LIMITED TECNO KA7             67
Samsung SM-A520F                           63
iPhone9,3                                  62
Samsung SM-G960F                           61
TECNO-J8                                   60
HUAWEI KOB-L09                             59
Samsung SM-G950F                           56
Samsung SM-G930F                           55
HUAWEI ANE-LX1                             55
TECNO K7                                   51
Itel S12                                   50
Samsung SM-G965F                           49
Samsung SM-A705FN                          48
Samsung SM-G935F                           46
Samsung SM-A105FN                          46
Samsung SM-A605FN                 

In [77]:
def def_device_name(name):
  if 'tecno' in name.lower():
    return 'tecno'
  if 'nokia' in name.lower():
    return 'nokia'
  if 'huawei' in name.lower():
    return 'huawei'
  if 'iphone' in name.lower():
    return 'iphone'
  if 'samsung' in name.lower():
    return 'samsung'
  if 'xiaomi' in name.lower():
    return 'xiaomi'
  if 'infinix' in name.lower():
    return 'infinix'
  if 'itel' in name.lower():
    return 'itel'
  return 'others'

In [78]:
df['device_group'] = df['device_name'].apply(def_device_name)
group_target('device_group').sort_values(by='mean',ascending=False)

,mean,count
device_group,,
tecno,0.167203,933
itel,0.133333,120
infinix,0.124060,266
nokia,0.113924,158
others,0.062331,369
samsung,0.038806,1675
iphone,0.021226,424
huawei,0.019444,720
xiaomi,0.011628,258


Получаем, что досточно огромная доля дешевых китайских телефонов (не huawei и xiaomi) связана с нашей целевой переменной. Скорее всего, у таких телефонов очень плохой gps-датчик.

In [79]:
df[df['device_group']=='others']['device_name'].value_counts()

HMD Global TA-1032    27
Foxconn TG-L800S      23
LGE LG-M200           11
Alps F9 Pro           10
OPPO A37fw             9
                      ..
ZTE BLADE A506         1
Asus ASUS_Z012D        1
Sony E5663             1
HTC U Ultra            1
Lava LAVA_R1           1
Name: device_name, Length: 115, dtype: int64

In [80]:
pd.pivot_table(data=df,index='device_group', columns='gps_confidence', values='overpaid_ride_ticket', aggfunc=['mean','count']).sort_values(by=('mean',0),ascending=False)

mean           count      
gps_confidence         0         1     0     1
device_group                                  
nokia           0.288889  0.044248    45   113
tecno           0.246944  0.104962   409   524
itel            0.235294  0.057971    51    69
infinix         0.211111  0.079545    90   176
samsung         0.172589  0.020974   197  1478
others          0.144444  0.035842    90   279
huawei          0.134615  0.010479    52   668
iphone          0.058824  0.017949    34   390
xiaomi          0.000000  0.012346    15   243

Гипотеза о том, что у дешевых китайских телефоном все плохо с gps, больше стала похожа на правду.  
Заметим также, что у телефонов Tecno с хорошим gps-сигналом процент относительно других довольно высок.  
Думаю, что это связано с малой способностью устройств этой марки справляться с необходимыми водителю задачами.

Посмотрим на влияние длительности и расстояния поездки на число переплат.  
Для этого создадим соотвествующие переменные:

*   Положительное значение переменных указывает на переплату со стороны компании
*   Отрицательное значение переменных указывает на переплату со стороны клиента



In [81]:
df['duration_error'] = df['duration'] - df['predicted_duration']
df['distance_error'] = df['distance'] - df['predicted_distance']
df['price_error'] = df['metered_price'] - df['upfront_price']

In [82]:
group_target('duration_error')

,mean,count
duration_error_bin,,
"(-20081.001, -122.0]",0.070994,986
"(-122.0, 34.0]",0.033401,988
"(34.0, 257.0]",0.027495,982
"(257.0, 845.0]",0.090539,983
"(845.0, 18086.0]",0.119919,984


In [83]:
group_target('distance_error')

,mean,count
distance_error_bin,,
"(-341860.001, -754.8]",0.072081,985
"(-754.8, 0.0]",0.065541,1007
"(0.0, 581.4]",0.028067,962
"(581.4, 2466.0]",0.033537,984
"(2466.0, 112012.0]",0.142132,985


При уменьшении ошибки расстояния увеличивается среднее число переплат.  
Относительно большая отрицательная ошибка в длительности и расстоянии возникает в следствие неправильной работы приложения, рассчитывающее цену клиенту.  

Огромная положительная ошибка длительности и расстояния указывает на проблему с построением оптимального маршрута в приложении-навигаторе водителя (плохо учитываются пробки или дорожные работы, например).

In [84]:
group_target('price_error')

,mean,count
price_error_bin,,
"(-581317.5410000001, -0.504]",0.052786,682
"(-0.504, 0.0]",0.009511,736
"(0.0, 0.89]",0.000000,629
"(0.89, 3.03]",0.007342,681
"(3.03, 178983.52]",0.088106,681


Переменная price_error подтверждает вышесказанные выводы.

# Выводы:

* переменные eu_indicator и prediction_price_type сильно коррелируют с числом переплат
* проблема с дальними поездками:
  * плохо прогнозируется цена в приложении клиента при дальней поездке
  * плохо строится маршрут (следовательно меняется факт. цена) при дальней поездке
* проблема некачественных мобильных устройств у водителей, используемых непосредственно в работе (выявлены марки "плохих" телефонов).

